In [1]:
!pip install -U imgaug==0.2.5
!pip install -U scipy==1.2.0

Requirement already up-to-date: imgaug==0.2.5 in /usr/local/lib/python3.6/dist-packages (0.2.5)
Requirement already up-to-date: scipy==1.2.0 in /usr/local/lib/python3.6/dist-packages (1.2.0)


In [0]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [0]:
import numpy as np

In [0]:
K.set_learning_phase(0)

W0617 05:54:07.771482 140556556978048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

W0617 05:54:07.811761 140556556978048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0617 05:54:07.817885 140556556978048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0617 05:54:07.847488 140556556978048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0617 05:54:07.851041 140556556978048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0617 05:54:10.215097 140556556978048 depre

In [0]:
# layer_contributions = {
#     'mixed2' : 0.2,
#     'mixed3' : 3.,
#     'mixed4' : 2.,
#     'mixed5' : 1.5,
# }
layer_contributions = {
    'mixed2' : 0.4,
    'mixed3' : 0.3,
    'mixed4' : 3.2,
    'mixed5' : 8.0,
}

In [0]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [0]:
loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

W0617 06:03:34.395488 140556556978048 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [0]:
dream = model.input

In [0]:
grads = K.gradients(loss, dream)[0]

In [0]:
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

In [0]:
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

In [0]:
def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

In [0]:
def gradient_ascent(x, iterations, step, max_loss = None):
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print('...loss value at', i, ':', loss_value)
    x += step*grad_values
  return x

In [0]:
step = 0.01
num_octave = 6
octave_scale = 1.4
iterations = 20
max_loss = 20

In [0]:
base_image_path = 'original_deep_dream.jpg'

In [0]:
img = preprocess_image(base_image_path)

In [0]:
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale**i)) for dim in original_shape])
  successive_shapes.append(shape)

In [0]:
successive_shapes = successive_shapes[::-1]

In [0]:
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [0]:
for shape in successive_shapes:
  print('preprocessing image shape', shape)
  img = resize_img(img, shape)
  img = gradient_ascent(
      img, 
      iterations = iterations,
      step = step,
      max_loss = max_loss
  )
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  same_size_original = resize_img(original_img, shape)
  lost_detail = same_size_original - upscaled_shrunk_original_img 
  img += lost_detail
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
save_img

preprocessing image shape (78, 118)
...loss value at 0 : 0.056639142
...loss value at 1 : 0.08329464
...loss value at 2 : 0.10457315
...loss value at 3 : 0.1261083
...loss value at 4 : 0.15030779
...loss value at 5 : 0.16964608
...loss value at 6 : 0.19405967
...loss value at 7 : 0.21162088
...loss value at 8 : 0.23195809
...loss value at 9 : 0.2538594
...loss value at 10 : 0.27483743
...loss value at 11 : 0.29206485
...loss value at 12 : 0.308375
...loss value at 13 : 0.32701656
...loss value at 14 : 0.34313473
...loss value at 15 : 0.35972318
...loss value at 16 : 0.37595809
...loss value at 17 : 0.38182297
...loss value at 18 : 0.40423074
...loss value at 19 : 0.41631454
preprocessing image shape (110, 166)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


...loss value at 0 : 0.55057
...loss value at 1 : 0.9671701
...loss value at 2 : 1.339853
...loss value at 3 : 1.6762469
...loss value at 4 : 2.096336
...loss value at 5 : 2.5415764
...loss value at 6 : 2.9560452
...loss value at 7 : 3.2381878
...loss value at 8 : 3.6056035
...loss value at 9 : 3.9320402
...loss value at 10 : 4.1699834
...loss value at 11 : 4.4206815
...loss value at 12 : 4.7148876
...loss value at 13 : 5.0977936
...loss value at 14 : 5.3428507
...loss value at 15 : 5.5447955
...loss value at 16 : 5.7182593
...loss value at 17 : 6.0820723
...loss value at 18 : 6.2848725
...loss value at 19 : 6.600898
preprocessing image shape (154, 233)


/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...loss value at 0 : 1.1016867
...loss value at 1 : 2.1259995
...loss value at 2 : 3.1021242
...loss value at 3 : 4.0458317
...loss value at 4 : 4.7619834
...loss value at 5 : 5.5718746
...loss value at 6 : 6.475348
...loss value at 7 : 7.198043
...loss value at 8 : 7.8608537
...loss value at 9 : 8.494375
...loss value at 10 : 9.376715
...loss value at 11 : 9.986687
...loss value at 12 : 10.951123
...loss value at 13 : 11.589012
...loss value at 14 : 11.891046
...loss value at 15 : 12.565638
...loss value at 16 : 13.206308
...loss value at 17 : 13.958692
...loss value at 18 : 14.387449
...loss value at 19 : 15.219765
preprocessing image shape (215, 326)
...loss value at 0 : 1.8394766
...loss value at 1 : 3.3269324
...loss value at 2 : 4.877241
...loss value at 3 : 6.54404
...loss value at 4 : 7.975411
...loss value at 5 : 9.550381
...loss value at 6 : 10.644568
...loss value at 7 : 12.204833
...loss value at 8 : 13.666934
...loss value at 9 : 15.729959
...loss value at 10 : 17.014458
.

<function __main__.save_img>

#auxiliary function

In [0]:
import scipy
import scipy.misc
from keras.preprocessing import image

In [0]:
def resize_img(img, size):
  img = np.copy(img)
  factors = (
      1,
      float(size[0])/img.shape[1],
      float(size[1])/img.shape[2],
      1
  )
  return scipy.ndimage.zoom(img, factors, order=1)

In [0]:
def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  scipy.misc.imsave(fname, pil_img)

In [0]:
def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

In [0]:
def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    x = x.reshape((x.shape[1], x.shape[2], 3))
  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x, 0, 255).astype('uint8')
  return x